In [244]:
#!pip3 install openai

import csv
import json
import openai
import os
import re
import time
import pandas as pd
import numpy as np

from os import path

openai.api_key = os.environ.get("OPENAI_API_KEY")

In [113]:
def get_schema(db_id):
    PATH = 'spider/database/'
    filename = 'schema.sql'
    
    path_to_file   = PATH + db_id + '/' + filename
    path_to_file_2 = PATH + db_id + '/' + db_id + '.sqlite'
    
    if path.exists(path_to_file): 
        return open(path_to_file, 'r').read()
    
    elif path.exists(path_to_file_2):
        return open(path_to_file_2, 'rb+').read()
    
    else:
        return None

def open_ai_prompt(x):
    return f'''Convert text to SQL.
        You have the following DDLs:
        ```
        {x['schema']}
        ```
        Write the SQL that answers the following question:
        """{x['question']}"""
        Respond with only one concise SQL statement.'''

def open_ai_completion(x):
    return f" {x['query']}\n"

def splitter(x: str):
    try:
        arr = x.split(';')
        create_tables = []
        for i, stmt in enumerate(arr):
            stmt = stmt.strip()
            if stmt.lower().startswith('create'):
                create_tables.append(stmt.replace('IF NOT EXISTS ', '').replace('\n', ''))
    except:
        arr = x.split(b';')
        create_tables = []
        for i, stmt in enumerate(arr):
            stmt = stmt.strip()
            if stmt.lower().startswith(b'create'):
                create_tables.append(stmt.replace('IF NOT EXISTS ', '').replace('\n', ''))
 
    return ';'.join(create_tables)

In [114]:
# Read in the first 100 Spider trining examples
df = pd.read_json('spider/train_spider.json')
df = df[['question', 'db_id', 'query']].head(100)

df['schema']         = df.apply(lambda x: get_schema(x['db_id']), axis=1)
df['schema']         = df['schema'].apply(splitter)
df['open_ai_prompt'] = df.apply(lambda x: open_ai_prompt(x), axis=1)
df['open_ai_completion'] = df.apply(lambda x: open_ai_completion(x), axis=1)
df.head()

,question,db_id,query,schema,open_ai_prompt,open_ai_completion
0,How many heads of the departments are older th...,department_management,SELECT count(*) FROM head WHERE age > 56,"CREATE TABLE ""department"" (""Department_ID"" int...",Convert text to SQL.\n You have the fol...,SELECT count(*) FROM head WHERE age > 56\n
1,"List the name, born state and age of the heads...",department_management,"SELECT name , born_state , age FROM head ORD...","CREATE TABLE ""department"" (""Department_ID"" int...",Convert text to SQL.\n You have the fol...,"SELECT name , born_state , age FROM head OR..."
2,"List the creation year, name and budget of eac...",department_management,"SELECT creation , name , budget_in_billions ...","CREATE TABLE ""department"" (""Department_ID"" int...",Convert text to SQL.\n You have the fol...,"SELECT creation , name , budget_in_billions..."
3,What are the maximum and minimum budget of the...,department_management,"SELECT max(budget_in_billions) , min(budget_i...","CREATE TABLE ""department"" (""Department_ID"" int...",Convert text to SQL.\n You have the fol...,"SELECT max(budget_in_billions) , min(budget_..."
4,What is the average number of employees of the...,department_management,SELECT avg(num_employees) FROM department WHER...,"CREATE TABLE ""department"" (""Department_ID"" int...",Convert text to SQL.\n You have the fol...,SELECT avg(num_employees) FROM department WHE...


# Pre-Fine Tuning
First, let's try explaining the problem to Da Vinci in English and see how it does on 100 Spider examples.

This gives the exact right answer 28% of the time.

In [166]:
davinci_raw = df[['open_ai_prompt','open_ai_completion']].copy().tail(90)

def call_davinci(row):
    prompt = row["open_ai_prompt"]
    
    # parameters chosen to match OpenAI Playground
    completions = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=256,
        stop=None,
        temperature=0.7,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    print(completions.choices[0].text.rstrip(";"))
    return completions.choices[0].text.rstrip(";")

davinci_raw["model_response"] = "" 
davinci_raw["model_response"] = davinci_raw.apply(call_davinci, axis=1)

davinci_raw['correct'] = np.where(davinci_raw['open_ai_completion'].str.strip().str.lower().apply(lambda x: re.sub('\s+', '',x)) ==
                                  davinci_raw['model_response'].str.strip().str.lower().apply(lambda x: re.sub('\s+', '',x)), 1, 0)



SELECT COUNT(DISTINCT temporary_acting) FROM management


SELECT COUNT(*) FROM department d LEFT JOIN management m ON d.Department_ID = m.Department_ID WHERE m.head_ID IS NULL


SELECT DISTINCT age FROM head JOIN management ON head.head_ID = management.head_ID WHERE temporary_acting = 'yes'


SELECT born_state
FROM head
INNER JOIN management
ON head.head_ID = management.head_ID
INNER JOIN department
ON management.department_ID = department.department_ID
WHERE department.name IN ('Treasury', 'Homeland Security')


SELECT Department_ID, Name, COUNT(head_ID) AS Num_Heads FROM department INNER JOIN management ON Department_ID = department_ID GROUP BY Department_ID, Name HAVING COUNT(head_ID) > 1


SELECT head_ID, name FROM head WHERE name LIKE '%Ha%'


SELECT COUNT(*) FROM farm


SELECT COUNT(*) FROM farm


SELECT SUM(Total_Horses) FROM farm ORDER BY SUM(Total_Horses) ASC


SELECT Farm_ID, SUM(Total_Horses) AS Total_Horses_Record FROM farm GROUP BY Farm_ID ORDER BY Total_Horses_Record AS



SELECT c.course_name, COUNT(s.student_id) AS 'Number of Registered Students'
FROM Courses c
INNER JOIN Student_Course_Registrations scr ON c.course_id = scr.course_id
INNER JOIN Students s ON s.student_id = scr.student_id
GROUP BY c.course_name


SELECT COUNT(Student_Course_Registrations.student_id),Courses.course_name 
FROM Student_Course_Registrations 
INNER JOIN Courses ON Student_Course_Registrations.course_id = Courses.course_id
GROUP BY Courses.course_id


SELECT candidate_id FROM Candidate_Assessments WHERE asessment_outcome_code = 'Pass'


SELECT candidate_id FROM Candidate_Assessments WHERE asessment_outcome_code = 'Pass'


SELECT cell_mobile_number 
FROM People 
INNER JOIN Candidates 
ON People.person_id = Candidates.candidate_id 
INNER JOIN Candidate_Assessments 
ON Candidates.candidate_id = Candidate_Assessments.candidate_id 
WHERE assessment_outcome_code = 'Fail'


SELECT cell_mobile_number 
FROM People 
INNER JOIN Candidates 
ON People.person_id = Candidates.candidate_i

In [167]:
# Correct on 4 of the first 10 queries
# Correct on 24 of the next 90 queries
# 28% accuracy
davinci_raw['correct'].mean()

0.26666666666666666

In [204]:
davinci_raw.iloc[0]['open_ai_prompt']

'Convert text to SQL.\n        You have the following DDLs:\n        ```\n        CREATE TABLE "department" ("Department_ID" int,"Name" text,"Creation" text,"Ranking" int,"Budget_in_Billions" real,"Num_Employees" real,PRIMARY KEY ("Department_ID"));CREATE TABLE "head" ("head_ID" int,"name" text,"born_state" text,"age" real,PRIMARY KEY ("head_ID"));CREATE TABLE "management" ("department_ID" int,"head_ID" int,"temporary_acting" text,PRIMARY KEY ("Department_ID","head_ID"),FOREIGN KEY ("Department_ID") REFERENCES `department`("Department_ID"),FOREIGN KEY ("head_ID") REFERENCES `head`("head_ID"))\n        ```\n        Write the SQL that answers the following question:\n        """How many acting statuses are there?"""\n        Respond with only one concise SQL statement.'

In [202]:
print(davinci_raw.iloc[0]['open_ai_completion'])
print(davinci_raw.iloc[0]['model_response'])

 SELECT count(DISTINCT temporary_acting) FROM management



SELECT COUNT(DISTINCT temporary_acting) FROM management


In [168]:
davinci_raw[davinci_raw['correct'] == 0]

,open_ai_prompt,open_ai_completion,model_response,correct
11,Convert text to SQL.\n You have the fol...,SELECT count(*) FROM department WHERE departm...,\n\nSELECT COUNT(*) FROM department d LEFT JOI...,0
12,Convert text to SQL.\n You have the fol...,SELECT DISTINCT T1.age FROM management AS T2 ...,\n\nSELECT DISTINCT age FROM head JOIN managem...,0
13,Convert text to SQL.\n You have the fol...,SELECT T3.born_state FROM department AS T1 JO...,\n\nSELECT born_state\nFROM head\nINNER JOIN m...,0
14,Convert text to SQL.\n You have the fol...,"SELECT T1.department_id , T1.name , count(*...","\n\nSELECT Department_ID, Name, COUNT(head_ID)...",0
18,Convert text to SQL.\n You have the fol...,SELECT Total_Horses FROM farm ORDER BY Total_...,\n\nSELECT SUM(Total_Horses) FROM farm ORDER B...,0
...,...,...,...,...
94,Convert text to SQL.\n You have the fol...,SELECT DISTINCT T1.city FROM addresses AS T1 ...,\n\nSELECT DISTINCT a.city\nFROM Addresses a\n...,0
95,Convert text to SQL.\n You have the fol...,SELECT course_name FROM courses ORDER BY cour...,\n\nSELECT course_name FROM Courses ORDER BY c...,0
96,Convert text to SQL.\n You have the fol...,SELECT course_name FROM courses ORDER BY cour...,\n\nSELECT course_name FROM Courses ORDER BY c...,0
97,Convert text to SQL.\n You have the fol...,SELECT first_name FROM people ORDER BY first_...,\n\nSELECT first_name FROM People ORDER BY fir...,0


In [170]:
print(re.sub('\s+', '', davinci_raw.loc[11]['open_ai_completion'].strip().lower()))
print(re.sub('\s+', '', davinci_raw.loc[11]['model_response'].strip().lower()))

selectcount(*)fromdepartmentwheredepartment_idnotin(selectdepartment_idfrommanagement);
selectcount(*)fromdepartmentdleftjoinmanagementmond.department_id=m.department_idwherem.head_idisnull


In [337]:
tmp = davinci_raw[~davinci_raw['open_ai_completion'].str.contains('JOIN')].copy()
print(tmp.shape)
print(tmp['correct'].mean())

(54, 4)
0.4444444444444444


In [345]:
i = 2
print(tmp[tmp['correct'] == 0].iloc[i]['open_ai_prompt'])
print(tmp[tmp['correct'] == 0].iloc[i]['open_ai_completion'])
print(tmp[tmp['correct'] == 0].iloc[i]['model_response'])

Convert text to SQL.
        You have the following DDLs:
        ```
        CREATE TABLE "city" ("City_ID" int,"Official_Name" text,"Status" text,"Area_km_2" real,"Population" real,"Census_Ranking" text,PRIMARY KEY ("City_ID"));CREATE TABLE "farm" ("Farm_ID" int,"Year" int,"Total_Horses" real,"Working_Horses" real,"Total_Cattle" real,"Oxen" real,"Bulls" real,"Cows" real,"Pigs" real,"Sheep_and_Goats" real,PRIMARY KEY ("Farm_ID"));CREATE TABLE "farm_competition" ("Competition_ID" int,"Year" int,"Theme" text,"Host_city_ID" int,"Hosts" text,PRIMARY KEY ("Competition_ID"),FOREIGN KEY (`Host_city_ID`) REFERENCES `city`(`City_ID`));CREATE TABLE "competition_record" ("Competition_ID" int,"Farm_ID" int,"Rank" int,PRIMARY KEY ("Competition_ID","Farm_ID"),FOREIGN KEY (`Competition_ID`) REFERENCES `farm_competition`(`Competition_ID`),FOREIGN KEY (`Farm_ID`) REFERENCES `farm`(`Farm_ID`))
        ```
        Write the SQL that answers the following question:
        """What is the total horses rec

# Fine Tune Da Vinci
Start with the same prompt as above

It's right in sample 12% of the time.
It's right out of sample 12% of the time.

How can those be exactly the same? And worse than the model without fine tuning?

In [ ]:
# data = []
# for idx, row in df.iterrows():
#     data.append({"prompt": row["open_ai_prompt"], "completion": row["open_ai_completion"]})

# with open("spider_open_ai_fine_tuning.jsonl", "w") as outfile:
#     for obj in data:
#         json.dump(obj, outfile)
#         outfile.write("\n")

In [183]:
#!openai tools fine_tunes.prepare_data -f spider_open_ai_fine_tuning.jsonl

Advice from OpenAI's prepare_data function:  
-For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.  
-All prompts end with suffix...This suffix seems very long. Consider replacing with a shorter suffix, such as `\n\n###\n\n`  
-All prompts start with prefix...Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion  
-Make sure to include `stop=["\n"]` so that the generated texts ends at the expected place.

In [ ]:
#!openai api fine_tunes.create -t "spider_open_ai_fine_tuning.jsonl" -m davinci

## In-Sample Testing

In [172]:
# model davinci:ft-mercator-2023-01-13-21-03-38 is trained on 100 prompts in long format with uncleaned schema
# testing here on the same data, but with the cleaned schema

df_is = df[['open_ai_prompt','open_ai_completion']].copy()

df_is["model_response"] = ""

def call_model_long_prompt(row):
    prompt = row["open_ai_prompt"]
    
    completions = openai.Completion.create(
        engine="davinci:ft-mercator-2023-01-13-21-03-38",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=["\n"],
        temperature=0.5
    )
    
    print(completions.choices[0].text)
    return completions.choices[0].text
    
df_is["model_response"] = df_is.apply(call_model_long_prompt, axis=1)

     SELECT count(*) FROM management WHERE age  >  56





     SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T3.born_state  =  'Alabama'







      SELECT T1.department_id ,  T1.name ,  count(*) FROM management AS T2 JOIN department AS T1 ON T1.department_id  =  T2.department_id GROUP BY T1.department_id HAVING count(*)  >  1







     SELECT Theme FROM farm_competition ORDER BY YEAR ASC


     SELECT avg(Working_Horses) FROM farm WHERE Total_Horses  >  5000

     SELECT max(Cows) ,  min(Cows) FROM farm





     SELECT Official_Name ,  Status FROM city ORDER BY Population DESC LIMIT 1

      SELECT T2.Year , T1.Official_Name FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID  =  T2.Host_city_ID
     SELECT T1.Official_Name FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID  =  T2.Host_city_ID GROUP BY T2.Host_city_ID HAVING COUNT(*)  >  1
        SEL

In [177]:
df_is['correct'] = np.where(df_is['open_ai_completion'].str
                                                       .strip()
                                                       .str
                                                       .lower()
                                                       .apply(lambda x: re.sub('\s+', '',x)) == 
                            df_is['model_response'].str
                                                   .strip()
                                                   .str
                                                   .lower()
                                                   .apply(lambda x: re.sub('\s+', '',x)), 1, 0)
print(df_is['correct'].mean())

0.12


In [178]:
df_is_wrong = df_is[df_is['correct'] == 0].copy().reset_index()

i = 0
# print(df_is_wrong.loc[i]['open_ai_prompt'])
print(df_is_wrong.loc[i]['open_ai_completion'])
print(df_is_wrong.loc[i]['model_response'])

 SELECT count(*) FROM head WHERE age  >  56

     SELECT count(*) FROM management WHERE age  >  56


## Out-of-Sample Testing

In [180]:
df_oos = pd.read_json('spider/train_spider.json')
df_oos = df_oos[['question', 'db_id', 'query']].tail(100)

df_oos['schema']         = df_oos.apply(lambda x: get_schema(x['db_id']), axis=1)
df_oos['schema']         = df_oos['schema'].apply(splitter)
df_oos['open_ai_prompt'] = df_oos.apply(lambda x: open_ai_prompt(x), axis=1)
df_oos['open_ai_completion'] = df_oos.apply(lambda x: open_ai_completion(x), axis=1)
df_oos.head()

,question,db_id,query,schema,open_ai_prompt,open_ai_completion
6900,Find the id of the customer who made the most ...,tracking_orders,SELECT T1.customer_id FROM customers AS T1 JOI...,CREATE TABLE `Customers` (`customer_id` INTEGE...,Convert text to SQL.\n You have the fol...,SELECT T1.customer_id FROM customers AS T1 JO...
6901,Give me a list of id and status of orders whic...,tracking_orders,"SELECT T2.order_id , T2.order_status FROM cus...",CREATE TABLE `Customers` (`customer_id` INTEGE...,Convert text to SQL.\n You have the fol...,"SELECT T2.order_id , T2.order_status FROM cu..."
6902,"Which orders are made by the customer named ""J...",tracking_orders,"SELECT T2.order_id , T2.order_status FROM cus...",CREATE TABLE `Customers` (`customer_id` INTEGE...,Convert text to SQL.\n You have the fol...,"SELECT T2.order_id , T2.order_status FROM cu..."
6903,Find the dates of orders which belong to the c...,tracking_orders,SELECT T2.date_order_placed FROM customers AS ...,CREATE TABLE `Customers` (`customer_id` INTEGE...,Convert text to SQL.\n You have the fol...,SELECT T2.date_order_placed FROM customers AS...
6904,What are the dates of the orders made by the c...,tracking_orders,SELECT T2.date_order_placed FROM customers AS ...,CREATE TABLE `Customers` (`customer_id` INTEGE...,Convert text to SQL.\n You have the fol...,SELECT T2.date_order_placed FROM customers AS...


In [181]:
df_oos["model_response"] = ""
df_oos["model_response"] = df_oos.apply(call_model_long_prompt, axis=1)

df_oos['correct'] = np.where(df_oos['open_ai_completion'].str
                                                         .strip()
                                                         .str
                                                         .lower()
                                                         .apply(lambda x: re.sub('\s+', '',x)) == 
                             df_oos['model_response'].str
                                                     .strip()
                                                     .str
                                                     .lower()
                                                     .apply(lambda x: re.sub('\s+', '',x)), 1, 0)
print(df_oos['correct'].mean())


      SELECT order_id , order_status FROM orders WHERE customer_id = 'Jeramie'



    """


      SELECT DISTINCT product_id FROM orders WHERE order_date BETWEEN '1975-01-01' AND '1976-01-01'






     SELECT date_order_placed FROM orders WHERE invoice_number = 10

     SELECT count(DISTINCT product_id) FROM orders GROUP BY product_id









     SELECT DISTINCT order_id FROM orders ORDER BY date_order_placed

     SELECT max(order_item_count) FROM orders

















     SELECT max(length_meters) FROM bridge JOIN architect ON bridge.architect_id  =  architect.id
     SELECT avg(length_feet) FROM bridge
     SELECT name ,  built_year FROM mill WHERE type = 'Grondzeiler'
        """SELECT DISTINCT T1.name , T1.nationality FROM architect AS T1 JOIN mill AS T2 ON T1.id  =  T2.architect_id GROUP BY T1.name , T1.nationality HAVING COUNT(*)  !=  2""

     SELECT DISTINCT T1.type FROM mill AS T1 JOIN architect AS T2 ON T1.architect_id  =  T2.id WHERE T2.nationality  =  'American' OR 

In [184]:
df_oos_wrong = df_oos[df_oos['correct'] == 0].copy().reset_index()

i = 0
print(df_oos_wrong.loc[i]['open_ai_prompt'])
print(df_oos_wrong.loc[i]['open_ai_completion'])
print(df_oos_wrong.loc[i]['model_response'])

Convert text to SQL.
        You have the following DDLs:
        ```
        CREATE TABLE `Customers` (`customer_id` INTEGER PRIMARY KEY,`customer_name` VARCHAR(80),`customer_details` VARCHAR(255));CREATE TABLE `Invoices` (`invoice_number` INTEGER PRIMARY KEY,`invoice_date` DATETIME,`invoice_details` VARCHAR(255));CREATE TABLE `Orders` (`order_id` INTEGER PRIMARY KEY,`customer_id` INTEGER NOT NULL,`order_status` VARCHAR(10) NOT NULL,`date_order_placed` DATETIME NOT NULL,`order_details` VARCHAR(255),FOREIGN KEY (`customer_id` ) REFERENCES `Customers`(`customer_id` ));CREATE TABLE `Products` (`product_id` INTEGER PRIMARY KEY,`product_name` VARCHAR(80),`product_details` VARCHAR(255));CREATE TABLE `Order_Items` (`order_item_id` INTEGER PRIMARY KEY,`product_id` INTEGER NOT NULL,`order_id` INTEGER NOT NULL,`order_item_status` VARCHAR(10) NOT NULL,`order_item_details` VARCHAR(255),FOREIGN KEY (`order_id` ) REFERENCES `Orders`(`order_id` ),FOREIGN KEY (`product_id` ) REFERENCES `Products`(`pr

# Fine Tune Da Vinci - Short Prompt
Follow OpenAI's advice on shortening the prompt used for fine tuning  
In sample accuracy is 91%.

In [185]:
def short_open_ai_prompt(x):
    return f'''Schema: {x['schema']}\nQuestion: {x['question']}\n\n###\n\n'''

In [187]:
df_short_is = pd.read_json('spider/train_spider.json')
df_short_is = df_short_is[['question', 'db_id', 'query']].head(100)

df_short_is['schema']         = df_short_is.apply(lambda x: get_schema(x['db_id']), axis=1)
df_short_is['schema']         = df_short_is['schema'].apply(splitter)
df_short_is['open_ai_prompt'] = df_short_is.apply(lambda x: short_open_ai_prompt(x), axis=1)
df_short_is['open_ai_completion'] = df_short_is.apply(lambda x: open_ai_completion(x), axis=1)

In [ ]:
# data = []
# for idx, row in df_short_is.iterrows():
#     data.append({"prompt": row["open_ai_prompt"], "completion": row["open_ai_completion"]})

# with open("spider_short_prompt_open_ai_fine_tuning.jsonl", "w") as outfile:
#     for obj in data:
#         json.dump(obj, outfile)
#         outfile.write("\n")

In [ ]:
#!openai tools fine_tunes.prepare_data -f spider_short_prompt_open_ai_fine_tuning.jsonl

In [ ]:
#!openai api fine_tunes.create -t "spider_short_prompt_open_ai_fine_tuning.jsonl" -m davinci

## In-Sample Testing

In [54]:
# model davinci:ft-mercator-2023-01-14-01-47-14 is trained on 100 prompts in short format with cleaned schema

# Testing on the last 10 in sample examples was 100% correct
# Testing on the first 90 in sample examples is 90% correct
# df_short_is_testing = df_short_is[['open_ai_prompt','open_ai_completion']].copy().head(90)

df_short_is["model_response"] = ""

def call_model_short(row):
    prompt = row["open_ai_prompt"]
    
    completions = openai.Completion.create(
        engine="davinci:ft-mercator-2023-01-14-01-47-14",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=["\n"],
        temperature=0.5
    )
    
    print(completions.choices[0].text)
    return completions.choices[0].text
    
df_short_is["model_response"] = df_short_is.apply(call_model_short, axis=1)

 SELECT count(*) FROM head WHERE age  >  56
 SELECT name ,  born_state ,  age FROM head ORDER BY age
 SELECT creation ,  name ,  budget_in_billions FROM department
 SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department
 SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15
 SELECT name FROM head WHERE born_state != 'California'
 SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T3.born_state  =  'Alabama'
 SELECT born_state FROM head GROUP BY born_state HAVING count(*)  >=  3
 SELECT creation FROM department GROUP BY creation ORDER BY count(*) DESC LIMIT 1
 SELECT T1.name ,  T1.num_employees FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id WHERE T2.temporary_acting  =  'Yes'
 SELECT count(DISTINCT temporary_acting) FROM management
 SELECT count(*) FROM department WHERE department_id NOT IN (SELECT d

 SELECT T1.course_name ,  count(*) FROM students AS T1 JOIN student_course_registrations AS T2 ON T1.student_id = T2.student_id GROUP BY T1.course_id
 SELECT candidate_id FROM candidate_assessments WHERE asessment_outcome_code = "Pass"
 SELECT candidate_id FROM candidate_assessments WHERE asessment_outcome_code = "Pass"
 SELECT T3.cell_mobile_number FROM candidates AS T1 JOIN candidate_assessments AS T2 ON T1.candidate_id = T2.candidate_id JOIN people AS T3 ON T1.candidate_id = T3.person_id WHERE T2.asessment_outcome_code = "Fail"
 SELECT T3.cell_mobile_number FROM candidates AS T1 JOIN candidate_assessments AS T2 ON T1.candidate_id = T2.candidate_id JOIN people AS T3 ON T1.candidate_id = T3.person_id WHERE T2.asessment_outcome_code = "Fail"
 SELECT student_id FROM student_course_registrations WHERE course_id  =  301
 SELECT student_id FROM student_course_attendance WHERE course_id  =  301
 SELECT student_id FROM student_course_attendance WHERE course_id = 301 ORDER BY date_of_attendan

In [55]:
df_short_is['correct'] = np.where(df_short_is['open_ai_completion'].str
                                                                   .strip()
                                                                   .str
                                                                   .lower()
                                                                   .apply(lambda x: re.sub('\s+', '',x)) == 
                                  df_short_is['model_response'].str
                                                               .strip()
                                                               .str
                                                               .lower()
                                                               .apply(lambda x: re.sub('\s+', '',x)), 1, 0)

df_short_is

,open_ai_prompt,open_ai_completion,model_response,correct
0,"Schema: CREATE TABLE ""department"" (""Department...",SELECT count(*) FROM head WHERE age > 56\n,SELECT count(*) FROM head WHERE age > 56,1
1,"Schema: CREATE TABLE ""department"" (""Department...","SELECT name , born_state , age FROM head OR...","SELECT name , born_state , age FROM head OR...",1
2,"Schema: CREATE TABLE ""department"" (""Department...","SELECT creation , name , budget_in_billions...","SELECT creation , name , budget_in_billions...",1
3,"Schema: CREATE TABLE ""department"" (""Department...","SELECT max(budget_in_billions) , min(budget_...","SELECT max(budget_in_billions) , min(budget_...",1
4,"Schema: CREATE TABLE ""department"" (""Department...",SELECT avg(num_employees) FROM department WHE...,SELECT avg(num_employees) FROM department WHE...,1
...,...,...,...,...
85,Schema: CREATE TABLE Addresses (address_id INT...,SELECT T3.cell_mobile_number FROM candidates ...,SELECT T3.cell_mobile_number FROM candidates ...,1
86,Schema: CREATE TABLE Addresses (address_id INT...,SELECT T3.cell_mobile_number FROM candidates ...,SELECT T3.cell_mobile_number FROM candidates ...,1
87,Schema: CREATE TABLE Addresses (address_id INT...,SELECT student_id FROM student_course_attenda...,SELECT student_id FROM student_course_registr...,0
88,Schema: CREATE TABLE Addresses (address_id INT...,SELECT student_id FROM student_course_attenda...,SELECT student_id FROM student_course_attenda...,1


In [56]:
df_short_is['correct'].mean()

0.9

In [61]:
df_short_is[df_short_is['correct'] == 0]

,open_ai_prompt,open_ai_completion,model_response,correct
36,"Schema: CREATE TABLE ""city"" (""City_ID"" int,""Of...",SELECT T1.Official_Name FROM city AS T1 JOIN ...,SELECT T1.Official_Name FROM city AS T1 JOIN ...,0
66,Schema: CREATE TABLE Addresses (address_id INT...,"SELECT T1.student_id , T2.course_name FROM s...","SELECT T1.student_id , T2.course_name FROM s...",0
71,Schema: CREATE TABLE Addresses (address_id INT...,SELECT count(*) FROM courses AS T1 JOIN stude...,SELECT count(*) FROM courses AS T1 JOIN stude...,0
72,Schema: CREATE TABLE Addresses (address_id INT...,SELECT count(*) FROM courses AS T1 JOIN stude...,SELECT count(*) FROM courses AS T1 JOIN stude...,0
77,Schema: CREATE TABLE Addresses (address_id INT...,SELECT T1.student_details FROM students AS T1...,SELECT T2.student_details FROM student_course...,0
78,Schema: CREATE TABLE Addresses (address_id INT...,SELECT T1.student_details FROM students AS T1...,SELECT T2.student_details FROM students AS T1...,0
81,Schema: CREATE TABLE Addresses (address_id INT...,"SELECT T3.course_name , count(*) FROM studen...","SELECT T1.course_name , count(*) FROM studen...",0
82,Schema: CREATE TABLE Addresses (address_id INT...,"SELECT T3.course_name , count(*) FROM studen...","SELECT T1.course_name , count(*) FROM studen...",0
87,Schema: CREATE TABLE Addresses (address_id INT...,SELECT student_id FROM student_course_attenda...,SELECT student_id FROM student_course_registr...,0


In [78]:
# Of the 9 that are wrong:
# No. 36 is close and possibly right but not exactly the same, depending on the data
# No. 66 is wrong
# No. 71 is wrong
# No. 72 is wrong
# No. 77 is written differently but is functionally identical (relabeled tables)
# No. 78 is wrong
# No. 81 is close and possibly right, depending on the data
# No. 82 is close and possibly right, depending on the data
# No. 87 is wrong
i = 87
print(df_short_is.iloc[i]['open_ai_prompt'])
print(df_short_is.iloc[i]['open_ai_completion'])
print(df_short_is.iloc[i]['model_response'])

Schema: CREATE TABLE Addresses (address_id INTEGER NOT NULL,line_1 VARCHAR(80),line_2 VARCHAR(80),city VARCHAR(50),zip_postcode CHAR(20),state_province_county VARCHAR(50),country VARCHAR(50),PRIMARY KEY (address_id));CREATE TABLE People (person_id INTEGER NOT NULL,first_name VARCHAR(255),middle_name VARCHAR(255),last_name VARCHAR(255),cell_mobile_number VARCHAR(40),email_address VARCHAR(40),login_name VARCHAR(40),password VARCHAR(40),PRIMARY KEY (person_id));CREATE TABLE Students (student_id INTEGER NOT NULL,student_details VARCHAR(255),PRIMARY KEY (student_id),FOREIGN KEY (student_id) REFERENCES People (person_id));CREATE TABLE Courses (course_id VARCHAR(100) NOT NULL,course_name VARCHAR(120),course_description VARCHAR(255),other_details VARCHAR(255),PRIMARY KEY (course_id));CREATE TABLE People_Addresses (person_address_id INTEGER NOT NULL,person_id INTEGER NOT NULL,address_id INTEGER NOT NULL,date_from DATETIME,date_to DATETIME,PRIMARY KEY (person_address_id),FOREIGN KEY (person_id) 

## Out-of-sample testing

In [190]:
df_short_oos = pd.read_json('spider/train_spider.json')
df_short_oos = df_short_oos[['question', 'db_id', 'query']].tail(100)

df_short_oos['schema']         = df_short_oos.apply(lambda x: get_schema(x['db_id']), axis=1)
df_short_oos['schema']         = df_short_oos['schema'].apply(splitter)
df_short_oos['open_ai_prompt'] = df_short_oos.apply(lambda x: short_open_ai_prompt(x), axis=1)
df_short_oos['open_ai_completion'] = df_short_oos.apply(lambda x: open_ai_completion(x), axis=1)

df_short_oos["model_response"] = ""    
df_short_oos["model_response"] = df_short_oos.apply(call_model_short, axis=1)

 SELECT customer_id FROM customers GROUP BY customer_id ORDER BY count(*) DESC LIMIT 1
 SELECT order_id , order_item_status FROM orders WHERE customer_name = 'Jeramie'
 SELECT order_id , order_status FROM orders WHERE customer_name = 'Jeramie'
 SELECT order_date FROM orders WHERE customer_name = "Jeramie"
 SELECT order_date FROM orders WHERE customer_name = 'Jeramie'
 SELECT T1.customer_name FROM customers AS T1 JOIN orders AS T2 ON T1.customer_id  =  T2.customer_id WHERE T2.date_order_placed  BETWEEN  '2009-01-01'  AND  '2010-01-01'
 SELECT T1.customer_name FROM customers AS T1 JOIN orders AS T2 ON T1.customer_id  =  T2.customer_id WHERE T1.customer_id  BETWEEN  2009 AND  2010
 SELECT DISTINCT PRODUCT_ID FROM ORDERS WHERE ORDER_DATE BETWEEN  1975-01-01 AND  1976-01-01
 SELECT DISTINCT T1.product_id FROM orders AS T1 JOIN order_items AS T2 ON T1.order_id  =  T2.order_id WHERE T1.date_order_placed  BETWEEN  1975-01-01  AND  1976-01-01
 SELECT T1.customer_name FROM customers AS T1 JOIN o

 SELECT count(*) FROM book_club GROUP BY publisher
 SELECT T1.publisher FROM book_club AS T1 JOIN book_club AS T2 ON T1.book_club_id  =  T2.book_club_id GROUP BY T1.publisher ORDER BY count(*) DESC LIMIT 1
 SELECT T1.publisher FROM book_club AS T1 JOIN book_club AS T2 ON T1.book_club_id  =  T2.book_club_id GROUP BY T1.publisher ORDER BY count(*) DESC LIMIT 1
 SELECT category ,  count(*) FROM book_club GROUP BY category
 SELECT count(DISTINCT Category) FROM book_club
 SELECT Category FROM book_club WHERE Year  >=  1989 GROUP BY Category HAVING COUNT(*)  >=  2
 SELECT T1.Category FROM book_club AS T1 JOIN book_club AS T2 ON T1.book_club_id  =  T2.book_club_id WHERE T2.Year  >  1989
 SELECT T1.Publisher FROM book_club AS T1 JOIN book_club AS T2 ON T1.book_club_id  =  T2.book_club_id WHERE T1.Year  =  1989 INTERSECT SELECT T1.Publisher FROM book_club AS T1 JOIN book_club AS T2 ON T1.book_club_id  =  T2.book_club_id WHERE T1.Year  =  1990
 SELECT Publisher FROM book_club WHERE Book_Title LI

In [191]:
df_short_oos['correct'] = np.where(df_short_oos['open_ai_completion'].str
                                                                     .strip()
                                                                     .str
                                                                     .lower()
                                                                     .apply(lambda x: re.sub('\s+', '',x)) == 
                                   df_short_oos['model_response'].str
                                                                 .strip()
                                                                 .str
                                                                 .lower()
                                                                 .apply(lambda x: re.sub('\s+', '',x)), 1, 0)

df_short_oos

,question,db_id,query,schema,open_ai_prompt,open_ai_completion,model_response,correct
6900,Find the id of the customer who made the most ...,tracking_orders,SELECT T1.customer_id FROM customers AS T1 JOI...,CREATE TABLE `Customers` (`customer_id` INTEGE...,Schema: CREATE TABLE `Customers` (`customer_id...,SELECT T1.customer_id FROM customers AS T1 JO...,SELECT customer_id FROM customers GROUP BY cu...,0
6901,Give me a list of id and status of orders whic...,tracking_orders,"SELECT T2.order_id , T2.order_status FROM cus...",CREATE TABLE `Customers` (`customer_id` INTEGE...,Schema: CREATE TABLE `Customers` (`customer_id...,"SELECT T2.order_id , T2.order_status FROM cu...","SELECT order_id , order_item_status FROM orde...",0
6902,"Which orders are made by the customer named ""J...",tracking_orders,"SELECT T2.order_id , T2.order_status FROM cus...",CREATE TABLE `Customers` (`customer_id` INTEGE...,Schema: CREATE TABLE `Customers` (`customer_id...,"SELECT T2.order_id , T2.order_status FROM cu...","SELECT order_id , order_status FROM orders WH...",0
6903,Find the dates of orders which belong to the c...,tracking_orders,SELECT T2.date_order_placed FROM customers AS ...,CREATE TABLE `Customers` (`customer_id` INTEGE...,Schema: CREATE TABLE `Customers` (`customer_id...,SELECT T2.date_order_placed FROM customers AS...,SELECT order_date FROM orders WHERE customer_...,0
6904,What are the dates of the orders made by the c...,tracking_orders,SELECT T2.date_order_placed FROM customers AS ...,CREATE TABLE `Customers` (`customer_id` INTEGE...,Schema: CREATE TABLE `Customers` (`customer_id...,SELECT T2.date_order_placed FROM customers AS...,SELECT order_date FROM orders WHERE customer_...,0
...,...,...,...,...,...,...,...,...
6995,What are all the company names that have a boo...,culture_company,SELECT T1.company_name FROM culture_company AS...,"CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...",SELECT T1.company_name FROM culture_company A...,SELECT T1.Company_name FROM book_club AS T1 J...,0
6996,Show the movie titles and book titles for all ...,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...","SELECT T1.title , T3.book_title FROM movie A...","SELECT T1.Title , T2.Title FROM movie AS T1 ...",0
6997,What are the titles of movies and books corres...,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...","CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...","SELECT T1.title , T3.book_title FROM movie A...",SELECT T1.Title FROM movie AS T1 JOIN book_cl...,0
6998,Show all company names with a movie directed i...,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,"CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...",SELECT T2.company_name FROM movie AS T1 JOIN ...,SELECT T1.Company_name FROM Company AS T1 JOI...,0


In [192]:
df_short_oos['correct'].mean()

0.12

In [193]:
df_short_oos[df_short_oos['correct'] == 1]

,question,db_id,query,schema,open_ai_prompt,open_ai_completion,model_response,correct
6926,Sort all the distinct products in alphabetical...,tracking_orders,SELECT DISTINCT product_name FROM products ORD...,CREATE TABLE `Customers` (`customer_id` INTEGE...,Schema: CREATE TABLE `Customers` (`customer_id...,SELECT DISTINCT product_name FROM products OR...,SELECT DISTINCT product_name FROM products OR...,1
6932,Find the name of the customer who made the mos...,tracking_orders,SELECT T1.customer_name FROM customers AS T1 J...,CREATE TABLE `Customers` (`customer_id` INTEGE...,Schema: CREATE TABLE `Customers` (`customer_id...,SELECT T1.customer_name FROM customers AS T1 ...,SELECT T1.customer_name FROM customers AS T1 ...,1
6948,What is the average length in feet of the brid...,architecture,SELECT avg(length_feet) FROM bridge,"CREATE TABLE ""architect"" (""id"" text,""name"" tex...","Schema: CREATE TABLE ""architect"" (""id"" text,""n...",SELECT avg(length_feet) FROM bridge\n,SELECT avg(length_feet) FROM bridge,1
6953,What are the ids and names of the architects w...,architecture,"SELECT T1.id , T1.name FROM architect AS T1 J...","CREATE TABLE ""architect"" (""id"" text,""name"" tex...","Schema: CREATE TABLE ""architect"" (""id"" text,""n...","SELECT T1.id , T1.name FROM architect AS T1 ...","SELECT T1.id , T1.name FROM architect AS T1 ...",1
6954,"What is the id, name and nationality of the ar...",architecture,"SELECT T1.id , T1.name , T1.nationality FROM...","CREATE TABLE ""architect"" (""id"" text,""name"" tex...","Schema: CREATE TABLE ""architect"" (""id"" text,""n...","SELECT T1.id , T1.name , T1.nationality FRO...","SELECT T1.id , T1.name , T1.nationality FRO...",1
6962,How many book clubs are there?,culture_company,SELECT count(*) FROM book_club,"CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...",SELECT count(*) FROM book_club\n,SELECT count(*) FROM book_club,1
6963,Count the number of book clubs.,culture_company,SELECT count(*) FROM book_club,"CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...",SELECT count(*) FROM book_club\n,SELECT count(*) FROM book_club,1
6964,"show the titles, and authors or editors for al...",culture_company,"SELECT book_title , author_or_editor FROM boo...","CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...","SELECT book_title , author_or_editor FROM bo...","SELECT Book_Title , Author_or_Editor FROM bo...",1
6968,"Show the years, book titles, and publishers fo...",culture_company,"SELECT YEAR , book_title , publisher FROM bo...","CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...","SELECT YEAR , book_title , publisher FROM b...","SELECT Year , Book_Title , Publisher FROM b...",1
6974,Show all book categories and the number of boo...,culture_company,"SELECT category , count(*) FROM book_club GRO...","CREATE TABLE ""book_club"" (""book_club_id"" int,""...","Schema: CREATE TABLE ""book_club"" (""book_club_i...","SELECT category , count(*) FROM book_club GR...","SELECT category , count(*) FROM book_club GR...",1


In [194]:
# 6990 has a pointless self-join but gives correct answer
# 6991 is the same as 6990
# 6994 is wrong and hallucinates a column
# 6995 is wrong and hallucinates a table
# 6996 is wrong
# 6997 answers half the question, replaces single quotes with double quotes
# 6998 is wrong

i = 6998
print(df_short_oos.loc[i]['open_ai_prompt'])
print(df_short_oos.loc[i]['open_ai_completion'])
print(df_short_oos.loc[i]['model_response'])

Schema: CREATE TABLE "book_club" ("book_club_id" int,"Year" int,"Author_or_Editor" text,"Book_Title" text,"Publisher" text,"Category" text,"Result" text,PRIMARY KEY ("book_club_id"));CREATE TABLE "movie" ("movie_id" int,"Title" text,"Year" int,"Director" text,"Budget_million" real,"Gross_worldwide" int,PRIMARY KEY("movie_id"));CREATE TABLE "culture_company" ("Company_name" text,"Type" text,"Incorporated_in" text,"Group_Equity_Shareholding" real,"book_club_id" text,"movie_id" text,PRIMARY KEY("Company_name"),FOREIGN KEY ("book_club_id") REFERENCES "book_club"("book_club_id"),FOREIGN KEY ("movie_id") REFERENCES "movie"("movie_id"))
Question: Show all company names with a movie directed in year 1999.

###


 SELECT T2.company_name FROM movie AS T1 JOIN culture_company AS T2 ON T1.movie_id  =  T2.movie_id WHERE T1.year  =  1999

 SELECT T1.Company_name FROM Company AS T1 JOIN movie AS T2 ON T1.company_name  =  T2.director_or_producer_name WHERE T2.year  =  '1999'


# Redo train-test split
Before, I took the first 100 samples as the training set and the last 100 as the test set. The samples aren't ordered randomly, though.  

Make sure to randomize at the level of schemas. Don't want to show it the schema in training and test on the same one.  
98% correct in sample  
16% correct out of sample

In [234]:
df2 = pd.read_json('spider/train_spider.json')
df_db_id = pd.DataFrame(df2['db_id'].unique(), columns=['db_id'])
df_db_id['train_test'] = np.random.choice(['train','test'], df_db_id.shape[0])
df2 = df2.merge(df_db_id, on='db_id')
df2.head(10)

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,train_test
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ...",test
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ...",test
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ...",test
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ...",test
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ...",test
5,department_management,SELECT name FROM head WHERE born_state != 'Cal...,"[SELECT, name, FROM, head, WHERE, born_state, ...","[select, name, from, head, where, born_state, ...",What are the names of the heads who are born o...,"[What, are, the, names, of, the, heads, who, a...","{'from': {'table_units': [['table_unit', 1]], ...",test
6,department_management,SELECT DISTINCT T1.creation FROM department AS...,"[SELECT, DISTINCT, T1.creation, FROM, departme...","[select, distinct, t1, ., creation, from, depa...",What are the distinct creation years of the de...,"[What, are, the, distinct, creation, years, of...","{'from': {'table_units': [['table_unit', 0], [...",test
7,department_management,SELECT born_state FROM head GROUP BY born_stat...,"[SELECT, born_state, FROM, head, GROUP, BY, bo...","[select, born_state, from, head, group, by, bo...",What are the names of the states where at leas...,"[What, are, the, names, of, the, states, where...","{'from': {'table_units': [['table_unit', 1]], ...",test
8,department_management,SELECT creation FROM department GROUP BY creat...,"[SELECT, creation, FROM, department, GROUP, BY...","[select, creation, from, department, group, by...",In which year were most departments established?,"[In, which, year, were, most, departments, est...","{'from': {'table_units': [['table_unit', 0]], ...",test
9,department_management,"SELECT T1.name , T1.num_employees FROM depart...","[SELECT, T1.name, ,, T1.num_employees, FROM, d...","[select, t1, ., name, ,, t1, ., num_employees,...",Show the name and number of employees for the ...,"[Show, the, name, and, number, of, employees, ...","{'from': {'table_units': [['table_unit', 0], [...",test


In [275]:
df2_is = df2[df2['train_test'] == 'train'][['question', 'db_id', 'query']].head(100)

df2_is['schema']         = df2_is.apply(lambda x: get_schema(x['db_id']), axis=1)
df2_is['schema']         = df2_is['schema'].apply(splitter)
df2_is['open_ai_prompt'] = df2_is.apply(lambda x: short_open_ai_prompt(x), axis=1)
df2_is['open_ai_completion'] = df2_is.apply(lambda x: open_ai_completion(x), axis=1)

In [242]:
df2_is

,question,db_id,query,schema,open_ai_prompt,open_ai_completion
56,which course has most number of registered stu...,student_assessment,SELECT T1.course_name FROM courses AS T1 JOIN ...,CREATE TABLE Addresses (address_id INTEGER NOT...,Schema: CREATE TABLE Addresses (address_id INT...,SELECT T1.course_name FROM courses AS T1 JOIN...
57,What is the name of the course with the most r...,student_assessment,SELECT T1.course_name FROM courses AS T1 JOIN ...,CREATE TABLE Addresses (address_id INTEGER NOT...,Schema: CREATE TABLE Addresses (address_id INT...,SELECT T1.course_name FROM courses AS T1 JOIN...
58,what is id of students who registered some cou...,student_assessment,SELECT student_id FROM student_course_registra...,CREATE TABLE Addresses (address_id INTEGER NOT...,Schema: CREATE TABLE Addresses (address_id INT...,SELECT student_id FROM student_course_registr...
59,What are the ids of the students who registere...,student_assessment,SELECT student_id FROM student_course_registra...,CREATE TABLE Addresses (address_id INTEGER NOT...,Schema: CREATE TABLE Addresses (address_id INT...,SELECT student_id FROM student_course_registr...
60,what are the first name and last name of all c...,student_assessment,"SELECT T2.first_name , T2.last_name FROM cand...",CREATE TABLE Addresses (address_id INTEGER NOT...,Schema: CREATE TABLE Addresses (address_id INT...,"SELECT T2.first_name , T2.last_name FROM can..."
...,...,...,...,...,...,...
151,What is the average longitude of stations that...,bike_1,SELECT avg(long) FROM station WHERE id NOT IN ...,CREATE TABLE station ( id INTEGER PRIMARY K...,Schema: CREATE TABLE station ( id INTEGER P...,SELECT avg(long) FROM station WHERE id NOT IN...
152,What is the mean longitude for all stations th...,bike_1,SELECT avg(long) FROM station WHERE id NOT IN ...,CREATE TABLE station ( id INTEGER PRIMARY K...,Schema: CREATE TABLE station ( id INTEGER P...,SELECT avg(long) FROM station WHERE id NOT IN...
153,When and in what zip code did max temperature ...,bike_1,"SELECT date , zip_code FROM weather WHERE max...",CREATE TABLE station ( id INTEGER PRIMARY K...,Schema: CREATE TABLE station ( id INTEGER P...,"SELECT date , zip_code FROM weather WHERE ma..."
154,What zip codes have a station with a max tempe...,bike_1,"SELECT date , zip_code FROM weather WHERE max...",CREATE TABLE station ( id INTEGER PRIMARY K...,Schema: CREATE TABLE station ( id INTEGER P...,"SELECT date , zip_code FROM weather WHERE ma..."


## Train

In [249]:
# Put the training data into jsonl format
data = []
for idx, row in df2_is.iterrows():
    data.append({"prompt": row["open_ai_prompt"], "completion": row["open_ai_completion"]})

timestr = time.strftime("%Y%m%d-%H%M%S")
with open("spider_open_ai_fine_tuning_" + timestr + ".jsonl", "w") as outfile:
    for obj in data:
        json.dump(obj, outfile)
        outfile.write("\n")

In [ ]:
#!openai tools fine_tunes.prepare_data -f spider_open_ai_fine_tuning_20230117-153523.jsonl

In [ ]:
#!openai api fine_tunes.create -t spider_open_ai_fine_tuning_20230117-153523.jsonl -m davinci

In [257]:
#!openai api fine_tunes.list

In [255]:
def call_model_2(row):
    prompt = row["open_ai_prompt"]
    
    completions = openai.Completion.create(
        engine="davinci:ft-mercator-2023-01-18-00-37-55",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=["\n"],
        temperature=0.5
    )
    
    print(completions.choices[0].text)
    return completions.choices[0].text

## Test in sample

In [256]:
df2_is["model_response"] = ""    
df2_is["model_response"] = df2_is.apply(call_model_2, axis=1)

 SELECT T1.course_name FROM courses AS T1 JOIN student_course_registrations AS T2 ON T1.course_id = T2.course_Id GROUP BY T1.course_id ORDER BY count(*) DESC LIMIT 1
 SELECT course_name FROM student_course_registrations GROUP BY course_id ORDER BY count(*) DESC LIMIT 1
 SELECT student_id FROM student_course_registrations GROUP BY student_id ORDER BY count(*) LIMIT 1
 SELECT student_id FROM student_course_registrations GROUP BY student_id ORDER BY count(*) LIMIT 1
 SELECT T2.first_name ,  T2.last_name FROM candidates AS T1 JOIN people AS T2 ON T1.candidate_id = T2.person_id
 SELECT T2.first_name ,  T2.last_name FROM candidates AS T1 JOIN people AS T2 ON T1.candidate_id = T2.person_id
 SELECT student_id FROM students WHERE student_id NOT IN (SELECT student_id FROM student_course_attendance)
 SELECT student_id FROM students WHERE student_id NOT IN (SELECT student_id FROM student_course_attendance)
 SELECT student_id FROM student_course_attendance
 SELECT student_id FROM student_course_att

 SELECT start_station_name ,  start_station_id FROM trip WHERE start_date LIKE "8/%" GROUP BY start_station_name ORDER BY COUNT(*) DESC LIMIT 1
 SELECT bike_id FROM trip WHERE zip_code  =  94002 GROUP BY bike_id ORDER BY COUNT(*) DESC LIMIT 1
 SELECT bike_id FROM trip WHERE zip_code  =  94002 GROUP BY bike_id ORDER BY COUNT(*) DESC LIMIT 1
 SELECT COUNT(*) FROM weather WHERE mean_humidity  >  50 AND mean_visibility_miles  >  8
 SELECT COUNT(*) FROM weather WHERE mean_humidity  >  50 AND mean_visibility_miles  >  8
 SELECT T1.lat ,  T1.long ,  T1.city FROM station AS T1 JOIN trip AS T2 ON T1.id  =  T2.start_station_id ORDER BY T2.duration LIMIT 1
 SELECT T1.lat ,  T1.long ,  T1.city FROM station AS T1 JOIN trip AS T2 ON T1.id  =  T2.start_station_id ORDER BY T2.duration LIMIT 1
 SELECT id FROM station WHERE city  =  "San Francisco" INTERSECT SELECT station_id FROM status GROUP BY station_id HAVING avg(bikes_available)  >  10
 SELECT id FROM station WHERE city  =  "San Francisco" INTERSE

In [258]:
df2_is['correct'] = np.where(df2_is['open_ai_completion'].str
                                                         .strip()
                                                         .str
                                                         .lower()
                                                         .apply(lambda x: re.sub('\s+', '',x)) == 
                             df2_is['model_response'].str
                                                     .strip()
                                                     .str
                                                     .lower()
                                                     .apply(lambda x: re.sub('\s+', '',x)), 1, 0)

In [259]:
df2_is['correct'].mean()

0.98

In [265]:
print(df2_is[df2_is['correct'] == 0].iloc[0]['open_ai_completion'])
print(df2_is[df2_is['correct'] == 0].iloc[0]['model_response'])

 SELECT T1.course_name FROM courses AS T1 JOIN student_course_registrations AS T2 ON T1.course_id = T2.course_Id GROUP BY T1.course_id ORDER BY count(*) DESC LIMIT 1

 SELECT course_name FROM student_course_registrations GROUP BY course_id ORDER BY count(*) DESC LIMIT 1


## Test out of sample

In [286]:
df2_oos = df2[df2['train_test'] == 'test'][['question', 'db_id', 'query']].head(100)

df2_oos['schema']         = df2_oos.apply(lambda x: get_schema(x['db_id']), axis=1)
df2_oos['schema']         = df2_oos['schema'].apply(splitter)
df2_oos['open_ai_prompt'] = df2_oos.apply(lambda x: short_open_ai_prompt(x), axis=1)
df2_oos['open_ai_completion'] = df2_oos.apply(lambda x: open_ai_completion(x), axis=1)

In [267]:
df2_oos["model_response"] = ""    
df2_oos["model_response"] = df2_oos.apply(call_model_2, axis=1)

 SELECT count(*) FROM management WHERE head_ID  >  56
 SELECT name , born_state , age FROM head ORDER BY age
 SELECT T1.Creation , T1.Name , T1.Budget_in_Billions FROM department AS T1 JOIN head AS T2 ON T1.Department_ID  =  T2.head_ID GROUP BY T1.Department_ID ORDER BY T1.Creation
 SELECT T1.Budget_in_Billions FROM department AS T1 JOIN management AS T2 ON T1.Department_ID = T2.department_ID GROUP BY T2.Department_ID ORDER BY SUM(T2.Budget_in_Billions) LIMIT 1 SELECT T1.Budget_in_Billions FROM department AS T1 JOIN management AS T2 ON T1.Department_ID = T2.department_ID GROUP BY T2.Department_ID ORDER BY MIN(T2.Budget_in_Billions)
 SELECT avg(num_employees) FROM department WHERE rank  BETWEEN 10 AND 15
 SELECT name FROM head WHERE born_state  NOT  LIKE  "California%"
 SELECT DISTINCT(creation) FROM department WHERE head_ID = (SELECT head_ID FROM management WHERE department_ID = (SELECT department_ID FROM department WHERE name = "Human Resources")) AND born_state = "Alabama"
 SELECT na

 SELECT DISTINCT T1.`catalog_entry_name` FROM `Catalog_Contents` AS T1 JOIN `Catalog_Entries` AS T2 ON T1.`parent_entry_id` = T2.`entry_id`
 SELECT T1.attribute_data_type FROM attribute_definitions AS T1 JOIN attribute_definitions AS T2 ON T1.attribute_id = T2.attribute_id JOIN attribute_definitions AS T3 ON T2.attribute_id = T3.attribute_id GROUP BY T2.attribute_id HAVING count(*) > 3
 SELECT attribute_name FROM attribute_definitions GROUP BY attribute_name HAVING COUNT(*) > 3
 SELECT attribute_data_type FROM attribute_definitions WHERE attribute_name = "Green"
 SELECT attribute_data_type FROM attribute_definitions WHERE attribute_name = "Green"
 SELECT T1.catalog_level_name ,  T2.catalog_level_number FROM catalog_structure AS T1 JOIN catalog_level_number AS T2 ON T1.catalog_id  =  T2.catalog_id WHERE T1.catalog_level_number  BETWEEN  5 AND 10
 SELECT T1.catalog_level_name , T1.catalog_level_number FROM catalog_structure WHERE catalog_level_number BETWEEN 5 AND 10
 SELECT T1.name FROM

In [269]:
df2_oos['correct'] = np.where(df2_oos['open_ai_completion'].str
                                                           .strip()
                                                           .str
                                                           .lower()
                                                           .apply(lambda x: re.sub('\s+', '',x)) == 
                              df2_oos['model_response'].str
                                                       .strip()
                                                       .str
                                                       .lower()
                                                       .apply(lambda x: re.sub('\s+', '',x)), 1, 0)

In [270]:
df2_oos['correct'].mean()

0.16

In [271]:
print(df2_oos[df2_oos['correct'] == 0].iloc[0]['open_ai_completion'])
print(df2_oos[df2_oos['correct'] == 0].iloc[0]['model_response'])

 SELECT count(*) FROM head WHERE age  >  56

 SELECT count(*) FROM management WHERE head_ID  >  56


# 100 -> 350 training examples
96% correct in sample  
23% correct out of sample  

Restrcited to single-table queries only:  
98% correct in sample
34% correct out of sample

In [276]:
df2_is_350 = df2[df2['train_test'] == 'train'][['question', 'db_id', 'query']].head(350)

df2_is_350['schema']         = df2_is_350.apply(lambda x: get_schema(x['db_id']), axis=1)
df2_is_350['schema']         = df2_is_350['schema'].apply(splitter)
df2_is_350['open_ai_prompt'] = df2_is_350.apply(lambda x: short_open_ai_prompt(x), axis=1)
df2_is_350['open_ai_completion'] = df2_is_350.apply(lambda x: open_ai_completion(x), axis=1)

In [277]:
# Put the training data into jsonl format
data = []
for idx, row in df2_is_350.iterrows():
    data.append({"prompt": row["open_ai_prompt"], "completion": row["open_ai_completion"]})

timestr = time.strftime("%Y%m%d-%H%M%S")
with open("spider_open_ai_fine_tuning_" + timestr + ".jsonl", "w") as outfile:
    for obj in data:
        json.dump(obj, outfile)
        outfile.write("\n")

In [ ]:
#!openai tools fine_tunes.prepare_data -f spider_open_ai_fine_tuning_20230117-174446.jsonl

In [ ]:
#!openai api fine_tunes.create -t spider_open_ai_fine_tuning_20230117-174446.jsonl -m davinci

In [279]:
!openai api fine_tunes.list

{
  "data": [
    {
      "created_at": 1672357089,
      "fine_tuned_model": "davinci:ft-mercator-2022-12-29-23-47-24",
      "hyperparams": {
        "batch_size": 1,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-ZMShnMrnhxzay9r0mXXEcxyI",
      "model": "davinci",
      "object": "fine-tune",
      "organization_id": "org-ePmgB4qVo14GgUKdUQci6IGz",
      "result_files": [
        {
          "bytes": 7017,
          "created_at": 1672357645,
          "filename": "compiled_results.csv",
          "id": "file-FdDaGVVXGWC08jN4NwodgYnY",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 3380,
          "created_at": 1672357089,
          "filename": "openai_classification_fine_tuning.txt",
          "id": "file-APupvHFP5Oae

In [298]:
def call_model_350(row):
    prompt = row["open_ai_prompt"]
    
    try:
        completions = openai.Completion.create(
            engine="davinci:ft-mercator-2023-01-18-02-13-35",
            prompt=prompt,
            max_tokens=1024,
            n=1,
            stop=["\n"],
            temperature=0.5
        )

        print(completions.choices[0].text)
        return completions.choices[0].text
    
    except:
        return None

## Test in sample

In [281]:
df2_is_350["model_response"] = ""    
df2_is_350["model_response"] = df2_is_350.apply(call_model_350, axis=1)

 SELECT T1.course_name FROM courses AS T1 JOIN student_course_registrations AS T2 ON T1.course_id = T2.course_id GROUP BY T1.course_id ORDER BY count(*) DESC LIMIT 1
 SELECT T1.course_name FROM courses AS T1 JOIN student_course_registrations AS T2 ON T1.course_id = T2.course_Id GROUP BY T1.course_id ORDER BY count(*) DESC LIMIT 1
 SELECT student_id FROM student_course_registrations GROUP BY student_id ORDER BY count(*) LIMIT 1
 SELECT student_id FROM student_course_registrations GROUP BY student_id ORDER BY count(*) LIMIT 1
 SELECT T1.first_name ,  T1.last_name FROM candidates AS T1 JOIN people AS T2 ON T1.candidate_id = T2.person_id
 SELECT T1.first_name ,  T1.last_name FROM candidates AS T1 JOIN people AS T2 ON T1.candidate_id = T2.person_id
 SELECT student_id FROM students WHERE student_id NOT IN (SELECT student_id FROM student_course_attendance)
 SELECT student_id FROM students WHERE student_id NOT IN (SELECT student_id FROM student_course_attendance)
 SELECT student_id FROM studen

 SELECT COUNT(*) FROM station WHERE city  =  "Mountain View"
 SELECT DISTINCT T1.name FROM station AS T1 JOIN status AS T2 ON T1.id  =  T2.station_id WHERE T2.bikes_available  =  7
 SELECT DISTINCT T1.name FROM station AS T1 JOIN status AS T2 ON T1.id  =  T2.station_id WHERE T2.bikes_available  =  7
 SELECT start_station_name ,  start_station_id FROM trip WHERE start_date LIKE "8/%" GROUP BY start_station_name ORDER BY COUNT(*) DESC LIMIT 1
 SELECT start_station_name ,  start_station_id FROM trip WHERE start_date LIKE "8/%" GROUP BY start_station_name ORDER BY COUNT(*) DESC LIMIT 1
 SELECT bike_id FROM trip WHERE zip_code  =  94002 GROUP BY bike_id ORDER BY COUNT(*) DESC LIMIT 1
 SELECT bike_id FROM trip WHERE zip_code  =  94002 GROUP BY bike_id ORDER BY COUNT(*) DESC LIMIT 1
 SELECT COUNT(*) FROM weather WHERE mean_humidity  >  50 AND mean_visibility_miles  >  8
 SELECT COUNT(*) FROM weather WHERE mean_humidity  >  50 AND mean_visibility_miles  >  8
 SELECT T1.lat ,  T1.long ,  T1.cit

 SELECT zip_code FROM weather GROUP BY zip_code HAVING avg(mean_humidity)  <  70 INTERSECT SELECT zip_code FROM trip GROUP BY zip_code HAVING count(*)  >=  100
 SELECT zip_code FROM weather GROUP BY zip_code HAVING avg(mean_humidity)  <  70 INTERSECT SELECT zip_code FROM trip GROUP BY zip_code HAVING count(*)  >=  100
 SELECT name FROM station WHERE city  =  "Palo Alto" EXCEPT SELECT end_station_name FROM trip GROUP BY end_station_name HAVING count(*)  >  100
 SELECT name FROM station WHERE city  =  "Palo Alto" EXCEPT SELECT end_station_name FROM trip GROUP BY end_station_name HAVING count(*)  >  100
 SELECT count(*) FROM station AS T1 JOIN trip AS T2 JOIN station AS T3 JOIN trip AS T4 ON T1.id  =  T2.start_station_id AND T2.id  =  T4.id AND T3.id  =  T4.end_station_id WHERE T1.city  =  "Mountain View" AND T3.city  =  "Palo Alto"
 SELECT count(*) FROM station AS T1 JOIN trip AS T2 JOIN station AS T3 JOIN trip AS T4 ON T1.id  =  T2.start_station_id AND T2.id  =  T4.id AND T3.id  =  T4.e

 SELECT eid ,  name FROM Employee ORDER BY salary DESC LIMIT 1
 SELECT eid ,  name FROM Employee ORDER BY salary DESC LIMIT 1
 SELECT name FROM Employee ORDER BY salary ASC LIMIT 3
 SELECT name FROM Employee ORDER BY salary ASC LIMIT 3
 SELECT name FROM Employee WHERE salary  >  (SELECT avg(salary) FROM Employee)
 SELECT name FROM Employee WHERE salary  >  (SELECT avg(salary) FROM Employee)
 SELECT eid ,  salary FROM Employee WHERE name  =  'Mark Young'
 SELECT eid ,  salary FROM Employee WHERE name  =  "Mark Young"
 SELECT count(*) FROM Flight
 SELECT count(*) FROM Flight
 SELECT flno ,  origin ,  destination FROM Flight ORDER BY origin
 SELECT flno ,  origin ,  destination FROM Flight ORDER BY origin
 SELECT flno FROM Flight WHERE origin  =  "Los Angeles"
 SELECT flno FROM Flight WHERE origin  =  "Los Angeles"
 SELECT origin FROM Flight WHERE destination  =  "Honolulu"
 SELECT origin FROM Flight WHERE destination  =  "Honolulu"
 SELECT departure_date ,  arrival_date FROM Flight WHERE

In [282]:
df2_is_350['correct'] = np.where(df2_is_350['open_ai_completion'].str
                                                         .strip()
                                                         .str
                                                         .lower()
                                                         .apply(lambda x: re.sub('\s+', '',x)) == 
                             df2_is_350['model_response'].str
                                                     .strip()
                                                     .str
                                                     .lower()
                                                     .apply(lambda x: re.sub('\s+', '',x)), 1, 0)

In [283]:
df2_is_350['correct'].mean()

0.9628571428571429

In [325]:
df2_is_350['open_ai_completion'].str.len().max()

322

In [297]:
df2_is_350['model_response'].str.len().max()

321

## Test out of sample

In [285]:
df2_oos_350 = df2[df2['train_test'] == 'test'][['question', 'db_id', 'query']].head(350)

df2_oos_350['schema']         = df2_oos_350.apply(lambda x: get_schema(x['db_id']), axis=1)
df2_oos_350['schema']         = df2_oos_350['schema'].apply(splitter)
df2_oos_350['open_ai_prompt'] = df2_oos_350.apply(lambda x: short_open_ai_prompt(x), axis=1)
df2_oos_350['open_ai_completion'] = df2_oos_350.apply(lambda x: open_ai_completion(x), axis=1)

In [299]:
df2_oos_350["model_response"] = ""    
df2_oos_350["model_response"] = df2_oos_350.apply(call_model_350, axis=1)

 SELECT COUNT(*) FROM head WHERE age  >  56
 SELECT T1.Name ,  T1.born_state ,  T1.age FROM head AS T1 JOIN department AS T2 ON T1.head_ID  =  T2.head_ID ORDER BY T1.age
 SELECT Creation ,  Name ,  Budget_in_Billions FROM department
 SELECT max(Budget_in_Billions) ,  min(Budget_in_Billions) FROM department
 SELECT avg(Num_Employees) FROM department WHERE Ranking BETWEEN 10 AND 15
 SELECT name FROM head WHERE born_state != "CA"
 SELECT DISTINCT Creation FROM department WHERE born_state  =  "Alabama"
 SELECT name FROM head GROUP BY born_state HAVING count(*)  >=  3
 SELECT YEAR(Creation) FROM department GROUP BY YEAR(Creation) ORDER BY COUNT(*) DESC LIMIT 1
 SELECT T2.Name ,  Num_Employees FROM department AS T1 JOIN management AS T2 ON T1.head_ID  =  T2.head_ID WHERE T1.temporary_acting  =  "Yes"
 SELECT count(*) FROM management WHERE temporary_acting != ''
 SELECT count(*) FROM head WHERE head_ID NOT IN (SELECT head_ID FROM management)
 SELECT DISTINCT age FROM management WHERE temporar

 SELECT T2.level_name FROM Catalog_Contents AS T1 JOIN Catalog_Contents_Additional_Attributes AS T2 ON T1.parent_entry_id = T2.catalog_entry_id GROUP BY T1.parent_entry_id ORDER BY T1.price_in_dollars DESC LIMIT 1
 SELECT T2.level_name FROM Catalog_Contents AS T1 JOIN Catalog_Contents_Additional_Attributes AS T2 ON T1.catalog_entry_id  =  T2.catalog_entry_id GROUP BY T1.catalog_entry_id ORDER BY T1.price_in_dollars DESC LIMIT 1
 SELECT avg(price_in_euros) ,  min(price_in_euros) FROM Products
 SELECT avg(price_in_euros) ,  min(price_in_euros) FROM Catalog_Contents
 SELECT product_stock_number FROM Catalog_Contents WHERE height  =  MAX(height)
 SELECT T1.catalog_entry_name FROM Catalog_Contents AS T1 JOIN Catalog_Contents_Additional_Attributes AS T2 ON T1.catalog_entry_id  =  T2.catalog_entry_id GROUP BY T2.catalog_entry_id ORDER BY T2.catalog_entry_id DESC LIMIT 1
 SELECT product_name FROM Catalog_Contents WHERE capacity  =  smallest(capacity)
 SELECT catalog_entry_name FROM Catalog_Con

 SELECT count(*) FROM Accounts AS T1 JOIN Customers AS T2 ON T1.customer_id = T2.customer_id WHERE T1.account_name LIKE "Debit%"
 SELECT count(DISTINCT card_type_code) FROM Customers_Cards WHERE card_type_code LIKE "Debit%"
 SELECT count(*) FROM Customers_Cards WHERE customer_id =  "Blanche Huels"
 SELECT count(*) FROM Customers AS T1 JOIN Customers_Cards AS T2 ON T1.customer_id = T2.customer_id WHERE T1.customer_first_name = "Blanche" AND T1.customer_last_name = "Huels"
 SELECT T1.customer_id ,  count(*) FROM Accounts AS T1 JOIN Customers AS T2 ON T1.customer_id  =  T2.customer_id GROUP BY T1.customer_id
 SELECT DISTINCT customer_id ,  count(*) FROM Customers_Cards GROUP BY customer_id
 SELECT customer_id ,  count(*) FROM Customers_Cards GROUP BY customer_id ORDER BY count(*) DESC LIMIT 1
 SELECT T1.customer_id ,  count(*) FROM customers AS T1 JOIN customers_cards AS T2 ON T1.customer_id = T2.customer_id GROUP BY T1.customer_id ORDER BY count(*) DESC LIMIT 1
 SELECT T1.customer_id ,  

In [301]:
df2_oos_350['model_response']

0             SELECT COUNT(*) FROM head WHERE age  >  56
1       SELECT T1.Name ,  T1.born_state ,  T1.age FRO...
2       SELECT Creation ,  Name ,  Budget_in_Billions...
3       SELECT max(Budget_in_Billions) ,  min(Budget_...
4       SELECT avg(Num_Employees) FROM department WHE...
                             ...                        
907     SELECT count(*) FROM Available_Policies WHERE...
908     SELECT count(*) FROM Available_Policies WHERE...
909     SELECT T1.Customer_name FROM Customers AS T1 ...
910     SELECT Customer_name FROM Customers GROUP BY ...
911     SELECT policy_type_code FROM Available_Polici...
Name: model_response, Length: 350, dtype: object

In [303]:
def regex(x):
    try: return re.sub('\s+', '', x)
    except: return None

In [304]:
df2_oos_350['correct'] = np.where(df2_oos_350['open_ai_completion'].str
                                                         .strip()
                                                         .str
                                                         .lower()
                                                         .apply(lambda x: re.sub('\s+', '',x)) == 
                             df2_oos_350['model_response'].str
                                                     .strip()
                                                     .str
                                                     .lower()
                                                     .apply(lambda x: regex(x)), 1, 0)

In [306]:
df2_oos_350['correct'].mean()

0.22857142857142856

In [313]:
print(df2_oos_350[df2_oos_350['correct'] == 0].iloc[0]['open_ai_prompt'])
print(df2_oos_350[df2_oos_350['correct'] == 0].iloc[0]['open_ai_completion'])
print(df2_oos_350[df2_oos_350['correct'] == 0].iloc[0]['model_response'])

Schema: CREATE TABLE "department" ("Department_ID" int,"Name" text,"Creation" text,"Ranking" int,"Budget_in_Billions" real,"Num_Employees" real,PRIMARY KEY ("Department_ID"));CREATE TABLE "head" ("head_ID" int,"name" text,"born_state" text,"age" real,PRIMARY KEY ("head_ID"));CREATE TABLE "management" ("department_ID" int,"head_ID" int,"temporary_acting" text,PRIMARY KEY ("Department_ID","head_ID"),FOREIGN KEY ("Department_ID") REFERENCES `department`("Department_ID"),FOREIGN KEY ("head_ID") REFERENCES `head`("head_ID"))
Question: List the name, born state and age of the heads of departments ordered by age.

###


 SELECT name ,  born_state ,  age FROM head ORDER BY age

 SELECT T1.Name ,  T1.born_state ,  T1.age FROM head AS T1 JOIN department AS T2 ON T1.head_ID  =  T2.head_ID ORDER BY T1.age


In [324]:
df2_oos_350[df2_oos_350['correct'] == 0].iloc[0]

question              List the name, born state and age of the heads...
db_id                                             department_management
query                 SELECT name ,  born_state ,  age FROM head ORD...
schema                CREATE TABLE "department" ("Department_ID" int...
open_ai_prompt        Schema: CREATE TABLE "department" ("Department...
open_ai_completion     SELECT name ,  born_state ,  age FROM head OR...
model_response         SELECT T1.Name ,  T1.born_state ,  T1.age FRO...
correct                                                               0
Name: 1, dtype: object

In [322]:
df2[df2['query'].str.len() == df2['query'].str.len().max()]['query'].iloc[0]

"SELECT T1.stu_fname FROM student AS T1 JOIN enroll AS T2 ON T1.stu_num  =  T2.stu_num JOIN CLASS AS T3 ON T2.class_code  =  T3.class_code JOIN course AS T4 ON T3.crs_code  =  T4.crs_code JOIN department AS T5 ON T5.dept_code  =  T4.dept_code WHERE T5.dept_name  =  'Accounting' INTERSECT SELECT T1.stu_fname FROM student AS T1 JOIN enroll AS T2 ON T1.stu_num  =  T2.stu_num JOIN CLASS AS T3 ON T2.class_code  =  T3.class_code JOIN course AS T4 ON T3.crs_code  =  T4.crs_code JOIN department AS T5 ON T5.dept_code  =  T4.dept_code WHERE T5.dept_name  =  'Computer Info. Systems'"

## Test only on single table queries

In [330]:
df2_is_350[~df2_is_350['open_ai_completion'].str.contains('JOIN')]['correct'].mean()

0.9802371541501976

In [331]:
df2_oos_350[~df2_oos_350['open_ai_completion'].str.contains('JOIN')]['correct'].mean()

0.34497816593886466